# NLP Project Real vs. Fake News

#### Adam Brenner

In [84]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB

In [4]:
fake_df = pd.read_csv("Fake.csv")
real_df = pd.read_csv("True.csv")

In [5]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [6]:
real_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


## Data Preprosessing

In [8]:
fake_df["type"] = "fake"
real_df["type"] = "real"

In [10]:
data = pd.concat([fake_df, real_df]).sample(frac = 1).reset_index(drop=True)

In [11]:
data['title'] = data['title'].str.lower()
data['text'] = data['text'].str.lower()
data['subject'] = data['subject'].str.lower()

In [12]:
data.drop(columns=["subject", "date"], inplace=True)

In [17]:
stop_words = stopwords.words('english')

In [90]:
def remove_stopwords_url(text):
    lst = re.findall('http://\S+|https://\S+', text)
    for i in lst:
        text = text.replace(i, '')
    temp_text = text.split(' ')
    temp_text = [i for i in temp_text if i not in stop_words]
    return ' '.join(temp_text)

In [91]:
data['title'] = data['title'].apply(remove_stopwords_url)

In [92]:
data['text'] = data['text'].apply(remove_stopwords_url)

In [93]:
data

,title,text,type
0,dem. rep. says steve bannon ‘stone cold racis...,"congressman hakeem jeffries says that, would w...",fake
1,conservative wants let scalia vote grave scot...,"obvious scheme keep supreme court slanted way,...",fake
2,say what? prosecutors claim convicted islamic ...,"according sun, george w. bush successor, bara...",fake
3,mexico's meade seek nomination ruling pri party,mexico city (reuters) - mexico outgoing financ...,real
4,factbox: irma vs andrew: 2017's big hurricane ...,(reuters) - florida governor rick scott warned...,real
...,...,...,...
44893,trump wanted attack lgbtq community; jared iv...,"earlier week, leaked draft executive orders ci...",fake
44894,factbox: trump twitter (sept 27) - filibuster ...,following statements were posted verified twit...,real
44895,"obama says faith cure fears stirred war, hard ...","washington (reuters) - president barack obama,...",real
44896,grew believing blacks could support democrats…...,keep eye anita moncreif knowledge power democr...,fake


## Machine Learning

In [94]:
from sklearn.model_selection import train_test_split

X = data["text"]
y = data["type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

### Linear SVC Classification

In [95]:
pipe1 = Pipeline([("tfidf", TfidfVectorizer()),
                 ("clf", LinearSVC())])

pipe1.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', LinearSVC())])

In [96]:
predictions1 = pipe1.predict(X_test)

In [97]:
from sklearn import metrics
metrics.accuracy_score(y_test, predictions1)

0.9972160356347439

After fitting the Linear SVC model to the train dataset and predicting on the test data we get an accuracy score of 99.7% which is very good. Therfore, we can predict real and fake news 99.7% of the time.

### SGDC Classification

In [98]:
pipe2 = Pipeline([("tfidf", TfidfVectorizer()),
                 ("clf", SGDClassifier())])

pipe2.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', SGDClassifier())])

In [99]:
predictions2 = pipe2.predict(X_test)

In [100]:
metrics.accuracy_score(y_test, predictions2)

0.9951002227171493

After fitting the SGDC Classification model to the train dataset and predicting on the test data we get an accuracy score of 99.5% which is very good. Therfore, we can predict real and fake news 99.5% of the time.

### Multinomial NB Classification

In [101]:
pipe3 = Pipeline([("tfidf", TfidfVectorizer()),
                 ("clf", MultinomialNB())])

pipe3.fit(X_train, y_train)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [102]:
predictions3 = pipe3.predict(X_test)

In [103]:
metrics.accuracy_score(y_test, predictions3)

0.9416481069042316

After fitting the Multinomial NB Classification model to the train dataset and predicting on the test data we get an accuracy score of 94.2% which is very good. Therfore, we can predict real and fake news 94.2% of the time.